In [2]:
import json
import sqlite3

In [9]:
# Charge le fichier JSON Merimée
filename = 'data/liste-des-immeubles-proteges-au-titre-des-monuments-historiques.json' 

with open(filename) as f:
    dataMerimee = json.load(f)
    
len(dataMerimee)
dataMerimee = [ d['fields'] for d in  dataMerimee ]

In [5]:
# Charge le fichier JSON OSM:
filename = 'data/osmTags_France.json' 

with open(filename) as f:
    dataOsm = json.load(f)
    
dataOsm = dataOsm['elements']
len(dataOsm)

33071

In [6]:
# Garde seulement les monuments avec le code commençant par PA...
dataOsm = [ el for el in dataOsm if el['tags']['ref:mhs'].startswith('PA') ]
len(dataOsm)

32246

In [7]:
# Creer une table dans une base sqlite
c = sqlite3.connect('data/data.db')

with c:
    cur = c.cursor() 
    c.execute('''DROP TABLE IF EXISTS merimee''')
    cur.execute("CREATE TABLE IF NOT EXISTS merimee(refmhs TEXT NOT NULL UNIQUE, name TEXT, region TEXT, \
                departement TEXT, commune TEXT, SCLE TEXT, DPRO TEXT, INSEE TEXT, STAT TEXT)")

In [14]:
for data in dataMerimee[:-1]:
    # print (data['REF'])
    cur.execute("INSERT INTO merimee VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?);",\
                (data['ref'], data['tico'], data['reg'], data['code_departement'], data['commune'],\
                data['scle'], data['dpro'], data['insee'], data['stat']))

IntegrityError: UNIQUE constraint failed: merimee.refmhs

In [125]:
c.execute('''DROP TABLE IF EXISTS osm''')
cur.execute("CREATE TABLE IF NOT EXISTS osm(refmhs TEXT NOT NULL, type TEXT, id INTEGER,\
            name TEXT, wikidata TEXT, wikipedia TEXT)")

In [126]:
for data in dataOsm['elements']:
    if 'wikidata' in data['tags']: 
        wkd = data['tags']['wikidata']
    else :
        wkd =""
    if 'name' in data['tags']:
        nom = data['tags']['name']
    else :
        nom = ""
    if 'wikipedia' in data['tags']:
        wp = data['tags']['wikipedia']
    else :
        wp =""
    cur.execute("INSERT INTO osm VALUES(?, ?, ?, ?, ?, ?);",\
                (data['tags']['ref:mhs'], data['id'], data['type'], nom,\
                 wkd, wp))

In [127]:
c.commit()
c.close()